In [2]:
import pandas as pd


In [3]:
df=pd.read_csv(r"E:\Project_1\asteroids_10000.csv")

In [4]:
df

,s.no,id,neo_reference_id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,1,2415949,2415949,415949 (2001 XY10),19.37,0.355267,0.794401,False,2024-01-02,57205.895120,0.337254,5.045241e+07,131.191622,Earth
1,2,3160747,3160747,(2003 SR84),26.00,0.016771,0.037501,False,2024-01-02,38589.054833,0.132343,1.979817e+07,51.481268,Earth
2,3,3309828,3309828,(2005 YQ96),20.62,0.199781,0.446725,True,2024-01-02,56413.014352,0.167013,2.498473e+07,64.967910,Earth
3,4,3457842,3457842,(2009 HC21),22.10,0.101054,0.225964,False,2024-01-02,21891.118219,0.492051,7.360980e+07,191.407879,Earth
4,5,3553062,3553062,(2010 XA11),26.10,0.016016,0.035813,False,2024-01-02,31468.978359,0.235802,3.527551e+07,91.727075,Earth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,3759669,3759669,(2016 SC),27.60,0.008027,0.017949,False,2025-08-25,43685.833909,0.228604,3.419861e+07,88.926787,Earth
9996,9997,3773985,3773985,(2017 HE),26.70,0.012149,0.027167,False,2025-08-25,32517.895766,0.491777,7.356878e+07,191.301210,Earth
9997,9998,3989082,3989082,(2020 BA5),24.10,0.040230,0.089958,False,2025-08-25,15354.726710,0.131358,1.965082e+07,51.098114,Earth
9998,9999,54051372,54051372,(2020 QX4),23.08,0.064351,0.143893,False,2025-08-25,44092.751686,0.290438,4.344895e+07,112.980492,Earth


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   s.no                               10000 non-null  int64  
 1   id                                 10000 non-null  int64  
 2   neo_reference_id                   10000 non-null  int64  
 3   name                               10000 non-null  object 
 4   absolute_magnitude_h               10000 non-null  float64
 5   estimated_diameter_min_km          10000 non-null  float64
 6   estimated_diameter_max_km          10000 non-null  float64
 7   is_potentially_hazardous_asteroid  10000 non-null  bool   
 8   close_approach_date                10000 non-null  object 
 9   relative_velocity_kmph             10000 non-null  float64
 10  astronomical                       10000 non-null  float64
 11  miss_distance_km                   10000 non-null  floa

In [6]:
# Ensure 'close_approach_date' is in DATE format
df['close_approach_date'] = pd.to_datetime(df['close_approach_date']).dt.date


In [7]:
import pymysql

# Establish connection to MySQL
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='raja',
    database='project_nasa'
)
cursor = connection.cursor()

# Create asteroids table (with index on 'id' field)
create_asteroids_table = """
CREATE TABLE IF NOT EXISTS asteroids (
    serial_no INT AUTO_INCREMENT PRIMARY KEY,
    id INT,
    name VARCHAR(255),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN,
    INDEX(id)  -- Add an index on the 'id' field for foreign key reference
);
"""

# Create close_approach table
create_close_approach_table = """
CREATE TABLE IF NOT EXISTS close_approach (
    id INT AUTO_INCREMENT PRIMARY KEY,
    neo_reference_id INT,  -- Foreign key field to link with 'id' in asteroids table
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(100),
    FOREIGN KEY (neo_reference_id) REFERENCES asteroids(id) ON DELETE CASCADE ON UPDATE CASCADE  -- Link 'id' field in asteroids table
);
"""

# Execute table creation
cursor.execute(create_asteroids_table)
cursor.execute(create_close_approach_table)

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Tables created successfully!")


Tables created successfully!


In [9]:
# Ensure 'df' is defined before proceeding
if 'df' in globals():
    # Split into two tables
    asteroids_df = df[['id', 'name', 'absolute_magnitude_h',
                       'estimated_diameter_min_km', 'estimated_diameter_max_km',
                       'is_potentially_hazardous_asteroid']].drop_duplicates()

    close_approach_df = df[['neo_reference_id', 'close_approach_date',
                            'relative_velocity_kmph', 'astronomical',
                            'miss_distance_km', 'miss_distance_lunar', 'orbiting_body']]
else:
    print("Error: 'df' is not defined. Please ensure the cell defining 'df' is executed.")


In [10]:
import pymysql

# Connect to MySQL
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='raja',
    database='project_nasa'
)
cursor = connection.cursor()

# Insert data into asteroids table
insert_asteroids = """
INSERT INTO asteroids (id, name, absolute_magnitude_h, estimated_diameter_min_km,
                       estimated_diameter_max_km, is_potentially_hazardous_asteroid)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in asteroids_df.iterrows():
    cursor.execute(insert_asteroids, tuple(row))

# Insert data into close_approach table
insert_close_approach = """
INSERT INTO close_approach (neo_reference_id, close_approach_date,
                            relative_velocity_kmph, astronomical,
                            miss_distance_km, miss_distance_lunar, orbiting_body)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for _, row in close_approach_df.iterrows():
    cursor.execute(insert_close_approach, tuple(row))

# Commit and close
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully into both tables!")


Data inserted successfully into both tables!


In [11]:
close_approach_df['close_approach_date'] = pd.to_datetime(close_approach_df['close_approach_date']).dt.date


C:\Users\mzraj\AppData\Local\Temp\ipykernel_8652\1755009959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_approach_df['close_approach_date'] = pd.to_datetime(close_approach_df['close_approach_date']).dt.date
